# Gandalf Pathfinding AI: Search Algorithms Implementation

## Project Objective:
### General Project Description and Problem Definition:
### Gandalf the Grey and Fellowship of the Ring

Gandalf the Grey is tasked with leading and guiding the Fellowship of the Ring to confront the Dark Lord Sauron. For this purpose, he must guide the Fellowship members through the orcs (Sauron's army) and deliver each to a specific position, finally reaching Gondor himself. Gandalf has a chess-like map that includes the following:
- Gandalf's starting point
- Position of each Fellowship member
- Position of orcs
- Location where each Fellowship member should be placed
- Final position (Gondor)

Each orc has a military rank that determines their surveillance range using Manhattan distance n, where Gandalf and Fellowship members cannot pass through. Since Gandalf moves at night when orcs are asleep, he can pass through surveilled cells without waking them, but only for n moves within their surveillance area. After n moves in an orc's surveillance area, they wake up. After leaving an orc's surveillance area, you can return to that area without considering previous moves.

### Input Format
Initial information is provided in a file with the following format:
- Line 1: n and m, where n is the number of rows and m is the number of columns
- Lines 2-3: Coordinates of start and end points respectively (row number from top, then column number from left)
- Line 4: k and l, where k is the number of orcs and l is the number of Fellowship members
- Next k lines: x, y, and c, where x is row number, y is column number, and c is the orc's military rank or Manhattan surveillance distance
- Next l lines: Current locations of Fellowship members in order
- Next l lines: Designated locations where Fellowship members should be placed in the same order. Note that each Fellowship member must be placed in a specific location, meaning the i-th Fellowship member must be placed in the i-th location.

In [ ]:
import heapq
import math
import time
from collections import deque

### Problem Modeling Approach
First, we define a Node class that includes the following attributes:
- state: The state of that node
- parent: The parent of that node in the search tree
- action: The action that caused this node to be created
- path_cost: The cost of reaching from initial state to this node
- fn: The value of function f(n) in A* algorithms for this node

In [ ]:
class Node:
    def __init__(self, state, parent, action, path_cost):
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.fn = 0

    def __lt__(self, other):
        return self.fn < other.fn

    def __gt__(self, other):
        return self.fn > other.fn
        
    def __hash__(self):
        return hash(self.state)
    
    def __eq__(self, other):
        return isinstance(other, Node) and self.state == other.state

In [ ]:
def actions(table, state, n, m):
    """Get possible actions from current state"""
    location = state[0]
    fellowships = list(state[1])
    fellowship_with_gandalf_end_location = state[2]
    possible_actions = []
    
    # Check if Gandalf can pick up a Fellowship member
    if len(fellowship_with_gandalf_end_location) == 0:
        for i, fellowship in enumerate(fellowships):
            if location == fellowship[0]:
                fellowship_with_gandalf_end_location = fellowship[1]
                fellowships.pop(i)
                break
    
    # Check if Gandalf can drop off Fellowship member
    if len(fellowship_with_gandalf_end_location) != 0:
        if location == fellowship_with_gandalf_end_location:
            fellowship_with_gandalf_end_location = ()
    
    # Check possible moves - optimized with dictionary
    moves = {'R': (0, 1), 'L': (0, -1), 'U': (-1, 0), 'D': (1, 0)}
    
    for action, (dr, dc) in moves.items():
        new_row, new_col = location[0] + dr, location[1] + dc
        if (0 <= new_row < n and 0 <= new_col < m and 
            table[new_row][new_col] != '*'):
            possible_actions.append(action)
    
    return possible_actions, tuple(fellowships), fellowship_with_gandalf_end_location

In [ ]:
def create_new_node(node, action, fellowship_with_gandalf_end_location):
    """Create new node based on action performed"""
    parent = node
    path_cost = node.path_cost + 1
    location = list(node.state[0])
    fellowships = node.state[1]
    
    # Apply action - optimized with dictionary lookup
    moves = {'R': (0, 1), 'L': (0, -1), 'U': (-1, 0), 'D': (1, 0)}
    dr, dc = moves[action]
    location[0] += dr
    location[1] += dc
    
    state = tuple(location), fellowships, fellowship_with_gandalf_end_location
    return Node(state, parent, action, path_cost)